In [35]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder
import joblib
import os

In [36]:
# 1. Veri Yükleme
data = pd.read_csv("male_players.csv")  # Verisetini yükle

C:\Users\ASUS\AppData\Local\Temp\ipykernel_23188\3161535642.py:2: DtypeWarning: Columns (108) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("male_players.csv")  # Verisetini yükle


In [37]:
# 2. Eksik Verileri Temizleme
data = data.dropna(subset=['value_eur'])  # Hedef sütunda eksik veri varsa kaldır
data.fillna(0, inplace=True)  # Diğer eksik verileri 0 ile doldur

In [38]:
# 3. Kullanılmayacak Sütunları Kaldır
unused_columns = [
                    'wage_eur','overall', 'potential','fifa_version', 'fifa_update', 'update_as_of', '' 'player_id', 'player_url', 'short_name', 'long_name', 'dob', 'club_team_id', 'club_name', 
                    'league_id', 'league_name', 'club_position', 'club_jersey_number', 'club_loaned_from', 
                    'club_joined_date', 'club_contract_valid_until_year', 'nationality_id', 'nationality_name', 
                    'player_tags', 'player_traits','ls', 'st', 'rs', 'lw', 'lf', 'cf', 'rf', 'rw', 
    'lam', 'cam', 'ram', 'lm', 'lcm', 'cm', 'rcm', 'rm', 'lwb', 'ldm', 'cdm', 'rdm', 'rwb', 'lb', 
    'lcb', 'cb', 'rcb', 'rb', 'gk','release_clause_eur', 'nation_team_id', 'nation_jersey_number','nation_position', 'body_type','league_level','real_face',]
data = data.drop(columns=unused_columns)

# Kullanılan sütunları yazdır
used_columns = data.columns
print("Kullanılan sütunlar:")
print(used_columns)

Kullanılan sütunlar:
Index(['player_positions', 'value_eur', 'age', 'height_cm', 'weight_kg',
       'preferred_foot', 'weak_foot', 'skill_moves',
       'international_reputation', 'work_rate', 'pace', 'shooting', 'passing',
       'dribbling', 'defending', 'physic', 'attacking_crossing',
       'attacking_finishing', 'attacking_heading_accuracy',
       'attacking_short_passing', 'attacking_volleys', 'skill_dribbling',
       'skill_curve', 'skill_fk_accuracy', 'skill_long_passing',
       'skill_ball_control', 'movement_acceleration', 'movement_sprint_speed',
       'movement_agility', 'movement_reactions', 'movement_balance',
       'power_shot_power', 'power_jumping', 'power_stamina', 'power_strength',
       'power_long_shots', 'mentality_aggression', 'mentality_interceptions',
       'mentality_positioning', 'mentality_vision', 'mentality_penalties',
       'mentality_composure', 'defending_marking_awareness',
       'defending_standing_tackle', 'defending_sliding_tackle',
     

In [39]:
# 4. "player_positions" sütunundaki değerlerden sadece ilk pozisyonu al
data['player_positions'] = data['player_positions'].apply(
    lambda x: x.split(',')[0].strip() if isinstance(x, str) else x
)

# 4.1 Kategorik Verileri Sayısal Değerlere Dönüştürme
categorical_columns = ['preferred_foot', 'work_rate', 'player_positions']
label_encoders = {}
for col in categorical_columns:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col].astype(str))  # Orijinal sütunu dönüştür
    label_encoders[col] = le

# 4.2 LabelEncoder karşılıklarını yazdır
for col, le in label_encoders.items():
    print(f"{col} için LabelEncoder değerleri:")
    label_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
    for key, value in label_mapping.items():
        print(f"  {key}: {value}")
    print("\n")

preferred_foot için LabelEncoder değerleri:
  Left: 0
  Right: 1


work_rate için LabelEncoder değerleri:
  High/High: 0
  High/Low: 1
  High/Medium: 2
  Low/High: 3
  Low/Low: 4
  Low/Medium: 5
  Medium/High: 6
  Medium/Low: 7
  Medium/Medium: 8


player_positions için LabelEncoder değerleri:
  CAM: 0
  CB: 1
  CDM: 2
  CF: 3
  CM: 4
  GK: 5
  LB: 6
  LM: 7
  LW: 8
  LWB: 9
  RB: 10
  RM: 11
  RW: 12
  RWB: 13
  ST: 14




In [40]:
print("Veri çerçevesindeki sütunlar:")
print(data.columns)

Veri çerçevesindeki sütunlar:
Index(['player_positions', 'value_eur', 'age', 'height_cm', 'weight_kg',
       'preferred_foot', 'weak_foot', 'skill_moves',
       'international_reputation', 'work_rate', 'pace', 'shooting', 'passing',
       'dribbling', 'defending', 'physic', 'attacking_crossing',
       'attacking_finishing', 'attacking_heading_accuracy',
       'attacking_short_passing', 'attacking_volleys', 'skill_dribbling',
       'skill_curve', 'skill_fk_accuracy', 'skill_long_passing',
       'skill_ball_control', 'movement_acceleration', 'movement_sprint_speed',
       'movement_agility', 'movement_reactions', 'movement_balance',
       'power_shot_power', 'power_jumping', 'power_stamina', 'power_strength',
       'power_long_shots', 'mentality_aggression', 'mentality_interceptions',
       'mentality_positioning', 'mentality_vision', 'mentality_penalties',
       'mentality_composure', 'defending_marking_awareness',
       'defending_standing_tackle', 'defending_sliding_tackl

In [42]:
##TÜM MODELLERİ TEK BİR YERDEN EĞİTEN FONKSİYON YA DA AŞAĞIDAN TEK TEK EĞİT
def train_models_by_position(data, label_encoders, position_list, model_save_dir='saved_models'):
    # Model dosyalarının kaydedileceği klasörü oluştur
    os.makedirs(model_save_dir, exist_ok=True)

    results = {}

    for position in position_list:
        print(f"\n{'='*50}")
        print(f"Pozisyon: {position}")

        # Pozisyonun sayısal karşılığı
        label = label_encoders['player_positions'].transform([position])[0]

        # Pozisyona göre filtreleme
        position_data = data[data['player_positions'] == label]

        # Yeterli veri yoksa atla
        if len(position_data) < 10:
            print(f"{position} pozisyonu için yeterli veri yok. Atlanıyor.")
            continue

        # Özellik ve hedef ayrımı
        X = position_data.drop(columns=['value_eur'])
        y = position_data['value_eur']

        # Eğitim ve test setlerine ayırma
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Model eğitimi
        model = RandomForestRegressor(random_state=42, n_estimators=100)
        model.fit(X_train, y_train)

        # Tahmin ve performans metrikleri
        y_pred = model.predict(X_test)
        mae = mean_absolute_error(y_test, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        r2 = r2_score(y_test, y_pred)

        print(f"MAE: {mae:.2f}")
        print(f"RMSE: {rmse:.2f}")
        print(f"R²: {r2:.4f}")

        # Feature importances
        importances = model.feature_importances_
        feature_names = X.columns
        sorted_indices = np.argsort(importances)[::-1]

        print(f"\n{position} için Feature Importances:")
        for i in sorted_indices:
            print(f"{feature_names[i]}: {importances[i]:.4f}")

        # Modeli .pkl olarak kaydet
        model_filename = os.path.join(model_save_dir, f"{position}_model.pkl")
        joblib.dump(model, model_filename)
        print(f"Model dosyası kaydedildi: {model_filename}")

        # Sonuçları sakla
        results[position] = {
            'model': model,
            'mae': mae,
            'rmse': rmse,
            'r2': r2,
            'importances': dict(zip(feature_names, importances)),
            'model_path': model_filename
        }

    return results

In [43]:
positions = ['CAM', 'CB', 'CDM', 'CF', 'CM', 'GK', 'LB', 'LM', 'LW', 'LWB',
             'RB', 'RM', 'RW', 'RWB', 'ST']

model_results = train_models_by_position(data, label_encoders, positions)



Pozisyon: CAM
MAE: 711893.34
RMSE: 2498287.28
R²: 0.8813

CAM için Feature Importances:
skill_ball_control: 0.5329
movement_reactions: 0.1040
dribbling: 0.0819
age: 0.0293
mentality_positioning: 0.0246
attacking_finishing: 0.0187
power_stamina: 0.0181
attacking_short_passing: 0.0179
passing: 0.0175
mentality_composure: 0.0174
skill_dribbling: 0.0128
skill_long_passing: 0.0084
movement_sprint_speed: 0.0077
shooting: 0.0075
mentality_vision: 0.0071
pace: 0.0065
defending_marking_awareness: 0.0058
power_shot_power: 0.0055
attacking_crossing: 0.0051
mentality_penalties: 0.0048
skill_fk_accuracy: 0.0047
mentality_aggression: 0.0045
movement_acceleration: 0.0044
height_cm: 0.0041
skill_curve: 0.0039
power_jumping: 0.0035
power_long_shots: 0.0033
attacking_heading_accuracy: 0.0033
attacking_volleys: 0.0028
defending_standing_tackle: 0.0027
movement_agility: 0.0024
goalkeeping_positioning: 0.0024
mentality_interceptions: 0.0023
goalkeeping_reflexes: 0.0021
physic: 0.0019
power_strength: 0.001

In [21]:
# ST pozisyonunun sayısal karşılığını al
st_label = label_encoders['player_positions'].transform(['ST'])[0]

# Sadece ST pozisyonundaki oyuncuları filtrele
st_data = data[data['player_positions'] == st_label]

# Özellik ve hedef ayrımı
X_st = st_data.drop(columns=['value_eur'])  # Özellikler
y_st = st_data['value_eur']  # Hedef

# Eğitim ve test setlerine ayırma
X_train_st, X_test_st, y_train_st, y_test_st = train_test_split(X_st, y_st, test_size=0.2, random_state=42)

# Model eğitimi ST
st_model = RandomForestRegressor(random_state=42, n_estimators=100)
st_model.fit(X_train_st, y_train_st)

# Tahmin ve performans değerlendirme ST
y_pred_st = st_model.predict(X_test_st)
mae_st = mean_absolute_error(y_test_st, y_pred_st)
rmse_st = np.sqrt(mean_squared_error(y_test_st, y_pred_st))
r2_st = r2_score(y_test_st, y_pred_st)

print(f"ST Model Performansı:")
print(f"MAE: {mae_st}")
print(f"RMSE: {rmse_st}")
print(f"R^2: {r2_st}")

# Feature importance ST
importances_st = st_model.feature_importances_
feature_names_st = X_st.columns

# Özellikleri önem sırasına göre sıralayalım
sorted_indices_st = np.argsort(importances_st)[::-1]

print("ST için Feature Importance:")
for i in sorted_indices_st:
    print(f"{feature_names_st[i]}: {importances_st[i]:.4f}")

ST Model Performansı:
MAE: 500390.51367840974
RMSE: 1574902.6518385687
R^2: 0.944244581184916
ST için Feature Importance:
shooting: 0.3597
attacking_finishing: 0.2166
movement_reactions: 0.1224
movement_sprint_speed: 0.0528
wage_eur: 0.0455
age: 0.0437
mentality_composure: 0.0195
skill_dribbling: 0.0135
skill_ball_control: 0.0115
dribbling: 0.0101
movement_acceleration: 0.0089
attacking_short_passing: 0.0083
mentality_positioning: 0.0066
attacking_heading_accuracy: 0.0053
physic: 0.0052
pace: 0.0052
power_strength: 0.0048
defending_marking_awareness: 0.0041
power_stamina: 0.0038
movement_agility: 0.0038
power_shot_power: 0.0034
skill_fk_accuracy: 0.0033
power_jumping: 0.0031
mentality_aggression: 0.0028
height_cm: 0.0023
attacking_crossing: 0.0023
mentality_penalties: 0.0022
mentality_interceptions: 0.0021
goalkeeping_reflexes: 0.0020
defending_standing_tackle: 0.0020
skill_long_passing: 0.0018
weight_kg: 0.0017
attacking_volleys: 0.0017
mentality_vision: 0.0017
movement_balance: 0.001

In [22]:
# GK pozisyonunun sayısal karşılığını al
gk_label = label_encoders['player_positions'].transform(['GK'])[0]

# Sadece GK pozisyonundaki oyuncuları filtrele
gk_data = data[data['player_positions'] == gk_label]

# Özellik ve hedef ayrımı
X_gk = gk_data.drop(columns=['value_eur'])  # Özellikler
y_gk = gk_data['value_eur']  # Hedef

# Eğitim ve test setlerine ayırma
X_train_gk, X_test_gk, y_train_gk, y_test_gk = train_test_split(X_gk, y_gk, test_size=0.2, random_state=42)

# Model eğitimi GK
gk_model = RandomForestRegressor(random_state=42, n_estimators=100)
gk_model.fit(X_train_gk, y_train_gk)

# Tahmin ve performans değerlendirme GK
y_pred_gk = gk_model.predict(X_test_gk)
mae_gk = mean_absolute_error(y_test_gk, y_pred_gk)
rmse_gk = np.sqrt(mean_squared_error(y_test_gk, y_pred_gk))
r2_gk = r2_score(y_test_gk, y_pred_gk)

print(f"GK Model Performansı:")
print(f"MAE: {mae_gk}")
print(f"RMSE: {rmse_gk}")
print(f"R^2: {r2_gk}")

# Feature importance GK
importances_gk = gk_model.feature_importances_
feature_names_gk = X_gk.columns

# Özellikleri önem sırasına göre sıralayalım
sorted_indices_gk = np.argsort(importances_gk)[::-1]

print("GK için Feature Importance:")
for i in sorted_indices_gk:
    print(f"{feature_names_gk[i]}: {importances_gk[i]:.4f}")

GK Model Performansı:
MAE: 351892.36768802226
RMSE: 1457433.1008845353
R^2: 0.9355079975590683
GK için Feature Importance:
goalkeeping_reflexes: 0.3834
goalkeeping_diving: 0.2326
movement_reactions: 0.0823
age: 0.0653
goalkeeping_handling: 0.0632
goalkeeping_positioning: 0.0515
wage_eur: 0.0213
power_shot_power: 0.0212
skill_dribbling: 0.0087
goalkeeping_kicking: 0.0069
mentality_composure: 0.0062
weak_foot: 0.0042
movement_agility: 0.0035
movement_balance: 0.0034
defending_marking_awareness: 0.0024
mentality_vision: 0.0023
skill_ball_control: 0.0023
mentality_interceptions: 0.0023
defending_sliding_tackle: 0.0023
weight_kg: 0.0021
attacking_heading_accuracy: 0.0021
skill_fk_accuracy: 0.0021
attacking_volleys: 0.0020
attacking_short_passing: 0.0020
movement_sprint_speed: 0.0019
movement_acceleration: 0.0018
height_cm: 0.0018
skill_long_passing: 0.0018
goalkeeping_speed: 0.0017
mentality_penalties: 0.0016
power_strength: 0.0016
mentality_positioning: 0.0016
power_jumping: 0.0014
mentali

In [ ]:
# CAM pozisyonunun sayısal karşılığını al
cam_label = label_encoders['player_positions'].transform(['CAM'])[0]

# Sadece CAM pozisyonundaki oyuncuları filtrele
cam_data = data[data['player_positions'] == cam_label]

# Özellik ve hedef ayrımı
X_cam = cam_data.drop(columns=['value_eur'])  # Özellikler
y_cam = cam_data['value_eur']  # Hedef

# Eğitim ve test setlerine ayırma
X_train_cam, X_test_cam, y_train_cam, y_test_cam = train_test_split(X_cam, y_cam, test_size=0.2, random_state=42)

# Model eğitimi CAM
cam_model = RandomForestRegressor(random_state=42, n_estimators=100)
cam_model.fit(X_train_cam, y_train_cam)

# Tahmin ve performans değerlendirme CAM
y_pred_cam = cam_model.predict(X_test_cam)
mae_cam = mean_absolute_error(y_test_cam, y_pred_cam)
rmse_cam = np.sqrt(mean_squared_error(y_test_cam, y_pred_cam))
r2_cam = r2_score(y_test_cam, y_pred_cam)

print(f"CAM Model Performansı:")
print(f"MAE: {mae_cam}")
print(f"RMSE: {rmse_cam}")
print(f"R^2: {r2_cam}")

# Feature importance CAM
importances_cam = cam_model.feature_importances_
feature_names_cam = X_cam.columns

# Özellikleri önem sırasına göre sıralayalım
sorted_indices_cam = np.argsort(importances_cam)[::-1]

print("CAM için Feature Importance:")
for i in sorted_indices_cam:
    print(f"{feature_names_cam[i]}: {importances_cam[i]:.4f}")


CAM Model Performansı:
MAE: 694424.0749306198
RMSE: 2361250.537276954
R^2: 0.8939909031301118
CAM için Feature Importance:
skill_ball_control: 0.5291
movement_reactions: 0.0819
dribbling: 0.0787
wage_eur: 0.0467
age: 0.0249
mentality_positioning: 0.0245
power_stamina: 0.0219
mentality_composure: 0.0191
attacking_finishing: 0.0184
attacking_short_passing: 0.0161
skill_dribbling: 0.0112
passing: 0.0100
skill_long_passing: 0.0075
shooting: 0.0074
mentality_vision: 0.0066
movement_sprint_speed: 0.0063
pace: 0.0058
defending: 0.0057
skill_fk_accuracy: 0.0056
movement_acceleration: 0.0052
height_cm: 0.0044
mentality_aggression: 0.0044
defending_marking_awareness: 0.0041
mentality_penalties: 0.0040
attacking_crossing: 0.0032
power_long_shots: 0.0032
power_jumping: 0.0031
power_shot_power: 0.0030
attacking_heading_accuracy: 0.0030
mentality_interceptions: 0.0029
skill_curve: 0.0028
defending_standing_tackle: 0.0027
attacking_volleys: 0.0024
movement_agility: 0.0023
movement_balance: 0.0023
ski

In [ ]:
# CB pozisyonunun sayısal karşılığını al
cb_label = label_encoders['player_positions'].transform(['CB'])[0]

# Sadece CB pozisyonundaki oyuncuları filtrele
cb_data = data[data['player_positions'] == cb_label]

# Özellik ve hedef ayrımı
X_cb = cb_data.drop(columns=['value_eur'])  # Özellikler
y_cb = cb_data['value_eur']  # Hedef

# Eğitim ve test setlerine ayırma
X_train_cb, X_test_cb, y_train_cb, y_test_cb = train_test_split(X_cb, y_cb, test_size=0.2, random_state=42)

# Model eğitimi CB
cb_model = RandomForestRegressor(random_state=42, n_estimators=100)
cb_model.fit(X_train_cb, y_train_cb)

# Tahmin ve performans değerlendirme CB
y_pred_cb = cb_model.predict(X_test_cb)
mae_cb = mean_absolute_error(y_test_cb, y_pred_cb)
rmse_cb = np.sqrt(mean_squared_error(y_test_cb, y_pred_cb))
r2_cb = r2_score(y_test_cb, y_pred_cb)

print(f"CB Model Performansı:")
print(f"MAE: {mae_cb}")
print(f"RMSE: {rmse_cb}")
print(f"R^2: {r2_cb}")

# Feature importance CB
importances_cb = cb_model.feature_importances_
feature_names_cb = X_cb.columns

# Özellikleri önem sırasına göre sıralayalım
sorted_indices_cb = np.argsort(importances_cb)[::-1]

print("CB için Feature Importance:")
for i in sorted_indices_cb:
    print(f"{feature_names_cb[i]}: {importances_cb[i]:.4f}")


CB Model Performansı:
MAE: 410206.00823045266
RMSE: 1250549.7816833917
R^2: 0.9434908736049902
CB için Feature Importance:
defending: 0.6195
movement_reactions: 0.0701
age: 0.0643
physic: 0.0363
mentality_composure: 0.0316
defending_standing_tackle: 0.0271
movement_sprint_speed: 0.0230
attacking_short_passing: 0.0140
wage_eur: 0.0113
pace: 0.0101
power_strength: 0.0072
attacking_volleys: 0.0060
skill_ball_control: 0.0057
dribbling: 0.0055
attacking_heading_accuracy: 0.0054
mentality_aggression: 0.0052
power_jumping: 0.0042
movement_acceleration: 0.0042
defending_marking_awareness: 0.0041
defending_sliding_tackle: 0.0029
mentality_penalties: 0.0024
skill_dribbling: 0.0022
power_stamina: 0.0021
mentality_interceptions: 0.0021
passing: 0.0021
movement_balance: 0.0020
mentality_vision: 0.0020
attacking_finishing: 0.0020
movement_agility: 0.0018
skill_long_passing: 0.0018
goalkeeping_kicking: 0.0018
power_long_shots: 0.0018
skill_fk_accuracy: 0.0017
mentality_positioning: 0.0017
goalkeeping

In [25]:
# CDM pozisyonunun sayısal karşılığını al
cdm_label = label_encoders['player_positions'].transform(['CDM'])[0]

# Sadece CDM pozisyonundaki oyuncuları filtrele
cdm_data = data[data['player_positions'] == cdm_label]

# Özellik ve hedef ayrımı
X_cdm = cdm_data.drop(columns=['value_eur'])  # Özellikler
y_cdm = cdm_data['value_eur']  # Hedef

# Eğitim ve test setlerine ayırma
X_train_cdm, X_test_cdm, y_train_cdm, y_test_cdm = train_test_split(X_cdm, y_cdm, test_size=0.2, random_state=42)

# Model eğitimi CDM
cdm_model = RandomForestRegressor(random_state=42, n_estimators=100)
cdm_model.fit(X_train_cdm, y_train_cdm)

# Tahmin ve performans değerlendirme CDM
y_pred_cdm = cdm_model.predict(X_test_cdm)
mae_cdm = mean_absolute_error(y_test_cdm, y_pred_cdm)
rmse_cdm = np.sqrt(mean_squared_error(y_test_cdm, y_pred_cdm))
r2_cdm = r2_score(y_test_cdm, y_pred_cdm)

print(f"CDM Model Performansı:")
print(f"MAE: {mae_cdm}")
print(f"RMSE: {rmse_cdm}")
print(f"R^2: {r2_cdm}")

# Feature importance CDM
importances_cdm = cdm_model.feature_importances_
feature_names_cdm = X_cdm.columns

# Özellikleri önem sırasına göre sıralayalım
sorted_indices_cdm = np.argsort(importances_cdm)[::-1]

print("CDM için Feature Importance:")
for i in sorted_indices_cdm:
    print(f"{feature_names_cdm[i]}: {importances_cdm[i]:.4f}")


CDM Model Performansı:
MAE: 532652.7914325843
RMSE: 1636075.0245824717
R^2: 0.9217222650481623
CDM için Feature Importance:
defending: 0.5280
movement_reactions: 0.0733
attacking_short_passing: 0.0505
wage_eur: 0.0497
mentality_composure: 0.0430
age: 0.0403
skill_ball_control: 0.0277
skill_dribbling: 0.0234
dribbling: 0.0208
power_long_shots: 0.0144
goalkeeping_kicking: 0.0083
skill_long_passing: 0.0082
defending_marking_awareness: 0.0065
mentality_vision: 0.0060
power_stamina: 0.0055
physic: 0.0052
attacking_volleys: 0.0052
passing: 0.0050
defending_standing_tackle: 0.0049
power_shot_power: 0.0046
mentality_interceptions: 0.0045
goalkeeping_handling: 0.0045
goalkeeping_positioning: 0.0041
mentality_penalties: 0.0041
goalkeeping_reflexes: 0.0038
goalkeeping_diving: 0.0033
weight_kg: 0.0032
attacking_heading_accuracy: 0.0030
shooting: 0.0028
mentality_aggression: 0.0028
defending_sliding_tackle: 0.0027
skill_fk_accuracy: 0.0026
movement_sprint_speed: 0.0025
mentality_positioning: 0.0024

In [26]:
# CF pozisyonunun sayısal karşılığını al
cf_label = label_encoders['player_positions'].transform(['CF'])[0]

# Sadece CF pozisyonundaki oyuncuları filtrele
cf_data = data[data['player_positions'] == cf_label]

# Özellik ve hedef ayrımı
X_cf = cf_data.drop(columns=['value_eur'])  # Özellikler
y_cf = cf_data['value_eur']  # Hedef

# Eğitim ve test setlerine ayırma
X_train_cf, X_test_cf, y_train_cf, y_test_cf = train_test_split(X_cf, y_cf, test_size=0.2, random_state=42)

# Model eğitimi CF
cf_model = RandomForestRegressor(random_state=42, n_estimators=100)
cf_model.fit(X_train_cf, y_train_cf)

# Tahmin ve performans değerlendirme CF
y_pred_cf = cf_model.predict(X_test_cf)
mae_cf = mean_absolute_error(y_test_cf, y_pred_cf)
rmse_cf = np.sqrt(mean_squared_error(y_test_cf, y_pred_cf))
r2_cf = r2_score(y_test_cf, y_pred_cf)

print(f"CF Model Performansı:")
print(f"MAE: {mae_cf}")
print(f"RMSE: {rmse_cf}")
print(f"R^2: {r2_cf}")

# Feature importance CF
importances_cf = cf_model.feature_importances_
feature_names_cf = X_cf.columns

# Özellikleri önem sırasına göre sıralayalım
sorted_indices_cf = np.argsort(importances_cf)[::-1]

print("CF için Feature Importance:")
for i in sorted_indices_cf:
    print(f"{feature_names_cf[i]}: {importances_cf[i]:.4f}")


CF Model Performansı:
MAE: 1307970.3125
RMSE: 3061038.991267551
R^2: 0.9096397790293577
CF için Feature Importance:
wage_eur: 0.3351
mentality_positioning: 0.1820
attacking_finishing: 0.0797
skill_ball_control: 0.0546
dribbling: 0.0456
skill_dribbling: 0.0433
shooting: 0.0359
attacking_short_passing: 0.0325
movement_reactions: 0.0310
age: 0.0174
power_stamina: 0.0102
pace: 0.0095
movement_sprint_speed: 0.0089
goalkeeping_diving: 0.0079
power_shot_power: 0.0068
passing: 0.0064
attacking_volleys: 0.0059
attacking_heading_accuracy: 0.0051
mentality_vision: 0.0048
power_jumping: 0.0048
skill_long_passing: 0.0048
skill_curve: 0.0044
skill_fk_accuracy: 0.0042
movement_agility: 0.0041
skill_moves: 0.0037
mentality_composure: 0.0036
mentality_penalties: 0.0035
work_rate: 0.0035
movement_acceleration: 0.0035
weak_foot: 0.0033
power_long_shots: 0.0032
goalkeeping_positioning: 0.0032
movement_balance: 0.0030
power_strength: 0.0028
defending_marking_awareness: 0.0025
mentality_aggression: 0.0023
d

In [ ]:
# CM pozisyonunun sayısal karşılığını al
cm_label = label_encoders['player_positions'].transform(['CM'])[0]

# Sadece CM pozisyonundaki oyuncuları filtrele
cm_data = data[data['player_positions'] == cm_label]

# Özellik ve hedef ayrımı
X_cm = cm_data.drop(columns=['value_eur'])  # Özellikler
y_cm = cm_data['value_eur']  # Hedef

# Eğitim ve test setlerine ayırma
X_train_cm, X_test_cm, y_train_cm, y_test_cm = train_test_split(X_cm, y_cm, test_size=0.2, random_state=42)

# Model eğitimi CM
cm_model = RandomForestRegressor(random_state=42, n_estimators=100)
cm_model.fit(X_train_cm, y_train_cm)

# Tahmin ve performans değerlendirme CM
y_pred_cm = cm_model.predict(X_test_cm)
mae_cm = mean_absolute_error(y_test_cm, y_pred_cm)
rmse_cm = np.sqrt(mean_squared_error(y_test_cm, y_pred_cm))
r2_cm = r2_score(y_test_cm, y_pred_cm)

print(f"CM Model Performansı:")
print(f"MAE: {mae_cm}")
print(f"RMSE: {rmse_cm}")
print(f"R^2: {r2_cm}")

# Feature importance CM
importances_cm = cm_model.feature_importances_
feature_names_cm = X_cm.columns

# Özellikleri önem sırasına göre sıralayalım
sorted_indices_cm = np.argsort(importances_cm)[::-1]

print("CM için Feature Importance:")
for i in sorted_indices_cm:
    print(f"{feature_names_cm[i]}: {importances_cm[i]:.4f}")


In [ ]:
# LB pozisyonunun sayısal karşılığını al
lb_label = label_encoders['player_positions'].transform(['LB'])[0]

# Sadece LB pozisyonundaki oyuncuları filtrele
lb_data = data[data['player_positions'] == lb_label]

# Özellik ve hedef ayrımı
X_lb = lb_data.drop(columns=['value_eur'])  # Özellikler
y_lb = lb_data['value_eur']  # Hedef

# Eğitim ve test setlerine ayırma
X_train_lb, X_test_lb, y_train_lb, y_test_lb = train_test_split(X_lb, y_lb, test_size=0.2, random_state=42)

# Model eğitimi LB
lb_model = RandomForestRegressor(random_state=42, n_estimators=100)
lb_model.fit(X_train_lb, y_train_lb)

# Tahmin ve performans değerlendirme LB
y_pred_lb = lb_model.predict(X_test_lb)
mae_lb = mean_absolute_error(y_test_lb, y_pred_lb)
rmse_lb = np.sqrt(mean_squared_error(y_test_lb, y_pred_lb))
r2_lb = r2_score(y_test_lb, y_pred_lb)

print(f"LB Model Performansı:")
print(f"MAE: {mae_lb}")
print(f"RMSE: {rmse_lb}")
print(f"R^2: {r2_lb}")

# Feature importance LB
importances_lb = lb_model.feature_importances_
feature_names_lb = X_lb.columns

# Özellikleri önem sırasına göre sıralayalım
sorted_indices_lb = np.argsort(importances_lb)[::-1]

print("LB için Feature Importance:")
for i in sorted_indices_lb:
    print(f"{feature_names_lb[i]}: {importances_lb[i]:.4f}")


In [ ]:
# LM pozisyonunun sayısal karşılığını al
lm_label = label_encoders['player_positions'].transform(['LM'])[0]

# Sadece LM pozisyonundaki oyuncuları filtrele
lm_data = data[data['player_positions'] == lm_label]

# Özellik ve hedef ayrımı
X_lm = lm_data.drop(columns=['value_eur'])  # Özellikler
y_lm = lm_data['value_eur']  # Hedef

# Eğitim ve test setlerine ayırma
X_train_lm, X_test_lm, y_train_lm, y_test_lm = train_test_split(X_lm, y_lm, test_size=0.2, random_state=42)

# Model eğitimi LM
lm_model = RandomForestRegressor(random_state=42, n_estimators=100)
lm_model.fit(X_train_lm, y_train_lm)

# Tahmin ve performans değerlendirme LM
y_pred_lm = lm_model.predict(X_test_lm)
mae_lm = mean_absolute_error(y_test_lm, y_pred_lm)
rmse_lm = np.sqrt(mean_squared_error(y_test_lm, y_pred_lm))
r2_lm = r2_score(y_test_lm, y_pred_lm)

print(f"LM Model Performansı:")
print(f"MAE: {mae_lm}")
print(f"RMSE: {rmse_lm}")
print(f"R^2: {r2_lm}")

# Feature importance LM
importances_lm = lm_model.feature_importances_
feature_names_lm = X_lm.columns

# Özellikleri önem sırasına göre sıralayalım
sorted_indices_lm = np.argsort(importances_lm)[::-1]

print("LM için Feature Importance:")
for i in sorted_indices_lm:
    print(f"{feature_names_lm[i]}: {importances_lm[i]:.4f}")


In [ ]:
# LW pozisyonunun sayısal karşılığını al
lw_label = label_encoders['player_positions'].transform(['LW'])[0]

# Sadece LW pozisyonundaki oyuncuları filtrele
lw_data = data[data['player_positions'] == lw_label]

# Özellik ve hedef ayrımı
X_lw = lw_data.drop(columns=['value_eur'])  # Özellikler
y_lw = lw_data['value_eur']  # Hedef

# Eğitim ve test setlerine ayırma
X_train_lw, X_test_lw, y_train_lw, y_test_lw = train_test_split(X_lw, y_lw, test_size=0.2, random_state=42)

# Model eğitimi LW
lw_model = RandomForestRegressor(random_state=42, n_estimators=100)
lw_model.fit(X_train_lw, y_train_lw)

# Tahmin ve performans değerlendirme LW
y_pred_lw = lw_model.predict(X_test_lw)
mae_lw = mean_absolute_error(y_test_lw, y_pred_lw)
rmse_lw = np.sqrt(mean_squared_error(y_test_lw, y_pred_lw))
r2_lw = r2_score(y_test_lw, y_pred_lw)

print(f"LW Model Performansı:")
print(f"MAE: {mae_lw}")
print(f"RMSE: {rmse_lw}")
print(f"R^2: {r2_lw}")

# Feature importance LW
importances_lw = lw_model.feature_importances_
feature_names_lw = X_lw.columns

# Özellikleri önem sırasına göre sıralayalım
sorted_indices_lw = np.argsort(importances_lw)[::-1]

print("LW için Feature Importance:")
for i in sorted_indices_lw:
    print(f"{feature_names_lw[i]}: {importances_lw[i]:.4f}")


In [ ]:
# LWB pozisyonunun sayısal karşılığını al
lwb_label = label_encoders['player_positions'].transform(['LWB'])[0]

# Sadece LWB pozisyonundaki oyuncuları filtrele
lwb_data = data[data['player_positions'] == lwb_label]

# Özellik ve hedef ayrımı
X_lwb = lwb_data.drop(columns=['value_eur'])  # Özellikler
y_lwb = lwb_data['value_eur']  # Hedef

# Eğitim ve test setlerine ayırma
X_train_lwb, X_test_lwb, y_train_lwb, y_test_lwb = train_test_split(X_lwb, y_lwb, test_size=0.2, random_state=42)

# Model eğitimi LWB
lwb_model = RandomForestRegressor(random_state=42, n_estimators=100)
lwb_model.fit(X_train_lwb, y_train_lwb)

# Tahmin ve performans değerlendirme LWB
y_pred_lwb = lwb_model.predict(X_test_lwb)
mae_lwb = mean_absolute_error(y_test_lwb, y_pred_lwb)
rmse_lwb = np.sqrt(mean_squared_error(y_test_lwb, y_pred_lwb))
r2_lwb = r2_score(y_test_lwb, y_pred_lwb)

print(f"LWB Model Performansı:")
print(f"MAE: {mae_lwb}")
print(f"RMSE: {rmse_lwb}")
print(f"R^2: {r2_lwb}")

# Feature importance LWB
importances_lwb = lwb_model.feature_importances_
feature_names_lwb = X_lwb.columns

# Özellikleri önem sırasına göre sıralayalım
sorted_indices_lwb = np.argsort(importances_lwb)[::-1]

print("LWB için Feature Importance:")
for i in sorted_indices_lwb:
    print(f"{feature_names_lwb[i]}: {importances_lwb[i]:.4f}")


In [ ]:
# RB pozisyonunun sayısal karşılığını al
rb_label = label_encoders['player_positions'].transform(['RB'])[0]

# Sadece RB pozisyonundaki oyuncuları filtrele
rb_data = data[data['player_positions'] == rb_label]

# Özellik ve hedef ayrımı
X_rb = rb_data.drop(columns=['value_eur'])  # Özellikler
y_rb = rb_data['value_eur']  # Hedef

# Eğitim ve test setlerine ayırma
X_train_rb, X_test_rb, y_train_rb, y_test_rb = train_test_split(X_rb, y_rb, test_size=0.2, random_state=42)

# Model eğitimi RB
rb_model = RandomForestRegressor(random_state=42, n_estimators=100)
rb_model.fit(X_train_rb, y_train_rb)

# Tahmin ve performans değerlendirme RB
y_pred_rb = rb_model.predict(X_test_rb)
mae_rb = mean_absolute_error(y_test_rb, y_pred_rb)
rmse_rb = np.sqrt(mean_squared_error(y_test_rb, y_pred_rb))
r2_rb = r2_score(y_test_rb, y_pred_rb)

print(f"RB Model Performansı:")
print(f"MAE: {mae_rb}")
print(f"RMSE: {rmse_rb}")
print(f"R^2: {r2_rb}")

# Feature importance RB
importances_rb = rb_model.feature_importances_
feature_names_rb = X_rb.columns

# Özellikleri önem sırasına göre sıralayalım
sorted_indices_rb = np.argsort(importances_rb)[::-1]

print("RB için Feature Importance:")
for i in sorted_indices_rb:
    print(f"{feature_names_rb[i]}: {importances_rb[i]:.4f}")


In [ ]:
# RM pozisyonunun sayısal karşılığını al
rm_label = label_encoders['player_positions'].transform(['RM'])[0]

# Sadece RM pozisyonundaki oyuncuları filtrele
rm_data = data[data['player_positions'] == rm_label]

# Özellik ve hedef ayrımı
X_rm = rm_data.drop(columns=['value_eur'])  # Özellikler
y_rm = rm_data['value_eur']  # Hedef

# Eğitim ve test setlerine ayırma
X_train_rm, X_test_rm, y_train_rm, y_test_rm = train_test_split(X_rm, y_rm, test_size=0.2, random_state=42)

# Model eğitimi RM
rm_model = RandomForestRegressor(random_state=42, n_estimators=100)
rm_model.fit(X_train_rm, y_train_rm)

# Tahmin ve performans değerlendirme RM
y_pred_rm = rm_model.predict(X_test_rm)
mae_rm = mean_absolute_error(y_test_rm, y_pred_rm)
rmse_rm = np.sqrt(mean_squared_error(y_test_rm, y_pred_rm))
r2_rm = r2_score(y_test_rm, y_pred_rm)

print(f"RM Model Performansı:")
print(f"MAE: {mae_rm}")
print(f"RMSE: {rmse_rm}")
print(f"R^2: {r2_rm}")

# Feature importance RM
importances_rm = rm_model.feature_importances_
feature_names_rm = X_rm.columns

# Özellikleri önem sırasına göre sıralayalım
sorted_indices_rm = np.argsort(importances_rm)[::-1]

print("RM için Feature Importance:")
for i in sorted_indices_rm:
    print(f"{feature_names_rm[i]}: {importances_rm[i]:.4f}")


In [ ]:
# RW pozisyonunun sayısal karşılığını al
rw_label = label_encoders['player_positions'].transform(['RW'])[0]

# Sadece RW pozisyonundaki oyuncuları filtrele
rw_data = data[data['player_positions'] == rw_label]

# Özellik ve hedef ayrımı
X_rw = rw_data.drop(columns=['value_eur'])  # Özellikler
y_rw = rw_data['value_eur']  # Hedef

# Eğitim ve test setlerine ayırma
X_train_rw, X_test_rw, y_train_rw, y_test_rw = train_test_split(X_rw, y_rw, test_size=0.2, random_state=42)

# Model eğitimi RW
rw_model = RandomForestRegressor(random_state=42, n_estimators=100)
rw_model.fit(X_train_rw, y_train_rw)

# Tahmin ve performans değerlendirme RW
y_pred_rw = rw_model.predict(X_test_rw)
mae_rw = mean_absolute_error(y_test_rw, y_pred_rw)
rmse_rw = np.sqrt(mean_squared_error(y_test_rw, y_pred_rw))
r2_rw = r2_score(y_test_rw, y_pred_rw)

print(f"RW Model Performansı:")
print(f"MAE: {mae_rw}")
print(f"RMSE: {rmse_rw}")
print(f"R^2: {r2_rw}")

# Feature importance RW
importances_rw = rw_model.feature_importances_
feature_names_rw = X_rw.columns

# Özellikleri önem sırasına göre sıralayalım
sorted_indices_rw = np.argsort(importances_rw)[::-1]

print("RW için Feature Importance:")
for i in sorted_indices_rw:
    print(f"{feature_names_rw[i]}: {importances_rw[i]:.4f}")


In [ ]:
# RWB pozisyonunun sayısal karşılığını al
rwb_label = label_encoders['player_positions'].transform(['RWB'])[0]

# Sadece RWB pozisyonundaki oyuncuları filtrele
rwb_data = data[data['player_positions'] == rwb_label]

# Özellik ve hedef ayrımı
X_rwb = rwb_data.drop(columns=['value_eur'])  # Özellikler
y_rwb = rwb_data['value_eur']  # Hedef

# Eğitim ve test setlerine ayırma
X_train_rwb, X_test_rwb, y_train_rwb, y_test_rwb = train_test_split(X_rwb, y_rwb, test_size=0.2, random_state=42)

# Model eğitimi RWB
rwb_model = RandomForestRegressor(random_state=42, n_estimators=100)
rwb_model.fit(X_train_rwb, y_train_rwb)

# Tahmin ve performans değerlendirme RWB
y_pred_rwb = rwb_model.predict(X_test_rwb)
mae_rwb = mean_absolute_error(y_test_rwb, y_pred_rwb)
rmse_rwb = np.sqrt(mean_squared_error(y_test_rwb, y_pred_rwb))
r2_rwb = r2_score(y_test_rwb, y_pred_rwb)

print(f"RWB Model Performansı:")
print(f"MAE: {mae_rwb}")
print(f"RMSE: {rmse_rwb}")
print(f"R^2: {r2_rwb}")

# Feature importance RWB
importances_rwb = rwb_model.feature_importances_
feature_names_rwb = X_rwb.columns

# Özellikleri önem sırasına göre sıralayalım
sorted_indices_rwb = np.argsort(importances_rwb)[::-1]

print("RWB için Feature Importance:")
for i in sorted_indices_rwb:
    print(f"{feature_names_rwb[i]}: {importances_rwb[i]:.4f}")


In [96]:
# Rastgele bir futbolcu verisi oluştur
sample_player = {
    'player_positions': 1873,  # Örnek olarak '0' (LabelEncoder ile dönüştürülmüş bir değer)
    'overall': 55,
    'potential': 70,
    'wage_eur': 200000,
    'age': 27,
    'height_cm': 185,
    'weight_kg': 80,
    'preferred_foot': 1,  # Örnek olarak '1' (LabelEncoder ile dönüştürülmüş bir değer)
    'weak_foot': 4,
    'skill_moves': 4,
    'international_reputation': 3,
    'work_rate': 2,  # Örnek olarak '2' (LabelEncoder ile dönüştürülmüş bir değer)
    'pace': 85,
    'shooting': 80,
    'passing': 78,
    'dribbling': 82,
    'defending': 60,
    'physic': 75,
    'attacking_crossing': 70,
    'attacking_finishing': 85,
    'attacking_heading_accuracy': 75,
    'attacking_short_passing': 80,
    'attacking_volleys': 78,
    'skill_dribbling': 85,
    'skill_curve': 80,
    'skill_fk_accuracy': 75,
    'skill_long_passing': 78,
    'skill_ball_control': 85,
    'movement_acceleration': 85,
    'movement_sprint_speed': 84,
    'movement_agility': 80,
    'movement_reactions': 85,
    'movement_balance': 78,
    'power_shot_power': 85,
    'power_jumping': 75,
    'power_stamina': 80,
    'power_strength': 78,
    'power_long_shots': 80,
    'mentality_aggression': 70,
    'mentality_interceptions': 60,
    'mentality_positioning': 85,
    'mentality_vision': 80,
    'mentality_penalties': 75,
    'mentality_composure': 85,
    'defending_marking_awareness': 85,
    'defending_standing_tackle': 90,
    'defending_sliding_tackle': 85
}

sample_player_df = pd.DataFrame([sample_player])

# Model ile tahmin yap
predicted_value = model.predict(sample_player_df)
print(f"Tahmin edilen piyasa değeri (value_eur): €{predicted_value[0]:,.2f}")

Tahmin edilen piyasa değeri (value_eur): €260,700.00


In [78]:
sample_player = {
'player_positions': 1873, # Örneğin 'ST' pozisyonuna karşılık gelen kod
'overall': 88,
'potential': 90,
'wage_eur': 300000,
'age': 27,
'height_cm': 185,
'weight_kg': 80,
'preferred_foot': 1, # 1 = Sağ
'weak_foot': 4,
'skill_moves': 4,
'international_reputation': 4,
'work_rate': 2, # Örneğin "High/Low"
# Genel yetenekler
'pace': 92,
'shooting': 91,
'passing': 75,
'dribbling': 89,
'defending': 25,
'physic': 85,

# Attacking Attributes
'attacking_crossing': 65,
'attacking_finishing': 94,
'attacking_heading_accuracy': 87,
'attacking_short_passing': 78,
'attacking_volleys': 88,

# Skill Attributes
'skill_dribbling': 90,
'skill_curve': 83,
'skill_fk_accuracy': 70,
'skill_long_passing': 68,
'skill_ball_control': 89,

# Movement Attributes
'movement_acceleration': 90,
'movement_sprint_speed': 93,
'movement_agility': 84,
'movement_reactions': 89,
'movement_balance': 80,

# Power Attributes
'power_shot_power': 91,
'power_jumping': 85,
'power_stamina': 82,
'power_strength': 84,
'power_long_shots': 86,

# Mentality Attributes
'mentality_aggression': 68,
'mentality_interceptions': 22,
'mentality_positioning': 92,
'mentality_vision': 79,
'mentality_penalties': 88,
'mentality_composure': 90,

# Defending Attributes
'defending_marking_awareness': 18,
'defending_standing_tackle': 20,
'defending_sliding_tackle': 15
}

sample_player_df = pd.DataFrame([sample_player])

# Model ile tahmin yap
predicted_value = model.predict(sample_player_df)
print(f"Tahmin edilen piyasa değeri (value_eur): €{predicted_value[0]:,.2f}")

Tahmin edilen piyasa değeri (value_eur): €87,035,000.00


In [89]:
mbappe = {
'player_positions': 204, # LW (örneğin), LabelEncoder karşılığı
'overall': 91,
'potential': 94,
'wage_eur': 2_300_000,
'age': 40,
'height_cm': 182,
'weight_kg': 10,
'preferred_foot': 1, # Right: 1
'weak_foot': 4,
'skill_moves': 5,
'international_reputation': 5,
'work_rate': 1, # High/Low → örneğin 0
'pace': 97,
'shooting': 90,
'passing': 80,
'dribbling': 92,
'defending': 36,
'physic': 78,

# Attacking Attributes
'attacking_crossing': 78,
'attacking_finishing': 94,
'attacking_heading_accuracy': 73,
'attacking_short_passing': 86,
'attacking_volleys': 84,

# Skill Attributes
'skill_dribbling': 93,
'skill_curve': 80,
'skill_fk_accuracy': 69,
'skill_long_passing': 71,
'skill_ball_control': 92,

# Movement Attributes
'movement_acceleration': 97,
'movement_sprint_speed': 97,
'movement_agility': 93,
'movement_reactions': 93,
'movement_balance': 82,

# Power Attributes
'power_shot_power': 90,
'power_jumping': 88,
'power_stamina': 88,
'power_strength': 77,
'power_long_shots': 83,

# Mentality Attributes
'mentality_aggression': 64,
'mentality_interceptions': 38,
'mentality_positioning': 93,
'mentality_vision': 83,
'mentality_penalties': 84,
'mentality_composure': 88,

# Defending Attributes
'defending_marking_awareness': 2,
'defending_standing_tackle': 4,
'defending_sliding_tackle': 5
}

sample_player_df = pd.DataFrame([mbappe])

# Model ile tahmin yap
predicted_value = model.predict(sample_player_df)
print(f"Tahmin edilen piyasa değeri (value_eur): €{predicted_value[0]:,.2f}")

Tahmin edilen piyasa değeri (value_eur): €158,480,000.00
